In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


## 1. Загружаем данные из файлов

In [2]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

train.head(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2. Проверяем, нет ли пропусков в данных

In [3]:
print("Пропущенные значения в train.csv:")
print(train.isnull().sum())
print("\nПропущенные значения в test.csv:")
print(test.isnull().sum())

Пропущенные значения в train.csv:
label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

Пропущенные значения в test.csv:
pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 784, dtype: int64


## 3. Подготавливаем данные

### 3.1. Подготовка данных

In [4]:
X = train.drop('label', axis=1).values / 255.0
y = train['label'].values

### 3.2. Разделение данных на тренировочные и валидационные

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Создание модели

### 4.1. Установка устройства

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 4.2. Создание модели в стиле Pytorch

In [7]:
# Определение Dataset
class DigitDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = torch.tensor(images, dtype=torch.float32).reshape(-1, 1, 28, 28)
        self.labels = torch.tensor(labels, dtype=torch.long) if labels is not None else None
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if self.labels is not None:
            return self.images[idx], self.labels[idx]
        else:
            return self.images[idx]

train_dataset = DigitDataset(X_train, y_train)
val_dataset = DigitDataset(X_val, y_val)
test_dataset = DigitDataset(test.values / 255.0)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Определение модели
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = DigitClassifier().to(device)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 5. Обучение модели

### 5.1. Обучение и валидация модели

In [8]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

    # Валидация модели
    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_predictions.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(val_labels, val_predictions)
    print(f'Validation Accuracy: {accuracy * 100:.2f}%')

Epoch 1, Loss: 0.23958630598549333
Validation Accuracy: 96.95%
Epoch 2, Loss: 0.06354765840495626
Validation Accuracy: 98.29%
Epoch 3, Loss: 0.043893254976497875
Validation Accuracy: 98.37%
Epoch 4, Loss: 0.033208444611657235
Validation Accuracy: 98.27%
Epoch 5, Loss: 0.02362095643422522
Validation Accuracy: 98.10%
Epoch 6, Loss: 0.019971291382708364
Validation Accuracy: 98.67%
Epoch 7, Loss: 0.013852220202104599
Validation Accuracy: 98.60%
Epoch 8, Loss: 0.011970645255210651
Validation Accuracy: 98.68%
Epoch 9, Loss: 0.010718141798445555
Validation Accuracy: 98.77%
Epoch 10, Loss: 0.010230629265896001
Validation Accuracy: 98.61%


## 6. Создание прогнозов для тестовых данных

In [9]:
model.eval()
test_predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_predictions.extend(predicted.cpu().numpy())

# Сохранение результатов
submission = pd.DataFrame({'ImageId': list(range(1, len(test_predictions) + 1)), 'Label': test_predictions})
submission.to_csv('submission.csv', index=False)